In [1]:
import re
import json
import os.path
import pandas as pd

from IPython.display import display
from tqdm import tqdm
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=0)

from gensim.models import Phrases
from gensim.models import LdaModel
from gensim.corpora import Dictionary

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import spacy
from spacy.tokens import Doc
print(spacy.__version__)
nlp = spacy.load("en_core_web_sm")

3.0.6


/opt/conda/lib/python3.8/site-packages/packaging/version.py:127: DeprecationWarning: Creating a LegacyVersion has been deprecated and will be removed in the next major release
  warnings.warn(


In [2]:
all_levels = pd.read_csv('all_levels.csv', low_memory=False)
print(len(all_levels))
display(all_levels.head())
# display(all_levels['topic'].value_counts()) # 128 prompts
texts = all_levels['text'].tolist()
print(len(texts))

# This elimates all student responses that are not automatically interpreted as strings by Pandas.This removes about 1,500 responses.
# A cursory investigation suggests that all these 'non-string' responses are NaN values, so useless.
texts = [text for text in texts if type(text) == str]

texts = [re.sub(r"\s+", r" ", text) for text in texts]

for i, text in enumerate(texts[0:20000]):
    if type(text) != str:
        print(text, i)
# print(texts[0:5])

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


549281


,Unnamed: 0,id,lvl,unit,author_id,author_nationality,topic,topic_id,date,grade,text
0,0,C18217,1,7,20967052,cn,Writing labels for a clothing store,22440,2012-04-20 21:12:14.890,79,Date:monday 11th. Time:9.30 am. From:Margar...
1,1,C18541,1,1,21016113,cn,Introducing yourself by email,3535,2011-12-24 03:50:49.100,85,"Dear teacher, My name's Yi Zhao,English is poo..."
2,2,C18648,1,1,20967075,cn,Introducing yourself by email,3535,2012-04-20 08:53:55.087,90,"My name's Henry Hong, I was born in Nanjing o..."
3,3,C20184,1,1,18898793,cn,Introducing yourself by email,3535,2011-12-14 08:54:48.380,95,"Dear Sir, I'm Jianwen Zhang, from a little tow..."
4,4,C20185,1,2,18898793,cn,Taking inventory in the office,9820,2011-12-26 09:49:48.140,98,"Dear Ms Thomas, There are thirteen computers a..."


549281


In [3]:
# Run corpus through the spaCy pipeline

def process_docs(texts):
    docs = list(tqdm(nlp.pipe(texts))) # 20 minutes or so...
    return docs

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
# Remove punctuation, stop words, and digits
# Make all words lowercase
# We can add to the stop words list here, or use other measures to eliminate uninteresting words (TF-IDF, Dispersion)

def reduce_tokens(docs):
    processed_docs = []
    for doc in docs:
        processed_doc = []
        for token in doc:
            if not token.is_punct and not token.is_stop and not token.is_digit:
                processed_doc.append(token.lemma_.lower())
        processed_docs.append(processed_doc)
    return processed_docs

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
# Compute bigrams.
# Add bigrams to docs (only ones that appear 20 times or more).

def compute_bigrams(processed_docs):
    bigram = Phrases(processed_docs, min_count=20)
    for idx in range(len(processed_docs)):
        for token in bigram[processed_docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                processed_docs[idx].append(token)
    return processed_docs

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
import os.path # Directories, gotta have em.

processed_docs_filename = "processed_docs.json"

if os.path.isfile(processed_docs_filename):
    print('Retrieving processed corpus from file')
    with open('processed_docs.json') as f:
        processed_docs = json.load(f)
    print('Retrieved')
else:
    print('Nothing found.')
    spacy_docs = process_docs(texts)
    reduced_docs = reduce_tokens(spacy_docs)
    processed_docs = compute_bigrams(reduced_docs)
    print('Corpus Processed.')
    

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Retrieving processed corpus from file
Retrieved


In [7]:
# Remove rare and common tokens.
# Create a dictionary representation of the documents.
dictionary = Dictionary(processed_docs)

# Filter out words that occur less than 20 documents, or more than 50% of the documents.
dictionary.filter_extremes(no_below=20, no_above=0.5)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Number of unique tokens: 24919
Number of documents: 547637


In [9]:
# Train LDA model.
# Set training parameters.
num_topics = 20
chunksize = 600000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
# print(model.print_topics())
# doc_lda = model[corpus]

p = gensimvis.prepare(model, corpus, dictionary)
p

/opt/conda/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3      0.153537 -0.014075       1        1  9.462536
7     -0.225808 -0.134781       2        1  8.875302
1     -0.295173 -0.049373       3        1  6.995727
11     0.068431 -0.090724       4        1  6.292627
10    -0.149739  0.281983       5        1  6.289962
0     -0.178544 -0.187714       6        1  5.631144
16     0.082354 -0.121302       7        1  5.445543
4      0.246179  0.031399       8        1  5.007173
12    -0.032610  0.345861       9        1  4.884635
14     0.046764  0.057640      10        1  4.384966
17     0.150580 -0.020638      11        1  4.226461
9     -0.027527 -0.025422      12        1  4.221115
19     0.023294 -0.007030      13        1  4.208124
15    -0.121756  0.072589      14        1  4.003777
5     -0.120932 -0.056420      15        1  3.906452
6     -0.004864  0.115503      16        1  3.775491
18     0.187761  0.012659      17        1  3.501422
8      0.032083 -0.060350      18        1  3.406023
13     0.082031 -0.102110      19        1  2.771060
2      0.083938 -0.047694      20        1  2.710458, topic_info=         Term           Freq          Total Category  logprob  loglift
3644  o'clock   55503.000000   55503.000000  Default  30.0000  30.0000
542         $   75345.000000   75345.000000  Default  29.0000  29.0000
272        go  112069.000000  112069.000000  Default  28.0000  28.0000
105      live  114146.000000  114146.000000  Default  27.0000  27.0000
357      hair   52091.000000   52091.000000  Default  26.0000  26.0000
...       ...            ...            ...      ...      ...      ...
20       dear    7335.127380   66910.906933  Topic20  -4.1618   1.3974
1214   sunday    5503.381767   28655.029827  Topic20  -4.4491   1.9581
521       pay    4894.754826   17311.590400  Topic20  -4.5663   2.3448
525       day    5960.302082  107802.197769  Topic20  -4.3694   0.7129
986       let    5121.768409   30048.314826  Topic20  -4.5210   1.8387

[1483 rows x 6 columns], token_table=       Topic      Freq                   Term
term                                         
542        1  0.000796                      $
542        2  0.000013                      $
542        3  0.000027                      $
542        5  0.008069                      $
542        7  0.000863                      $
...      ...       ...                    ...
3797       5  0.001156                    zoo
3797      17  0.843002                    zoo
20246      1  0.000252              zookeeper
20246     17  0.999285              zookeeper
23530     17  0.998040  zookeeper_flossington

[8389 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 8, 2, 12, 11, 1, 17, 5, 13, 15, 18, 10, 20, 16, 6, 7, 19, 9, 14, 3])